# Logistic Regression

In [136]:
import pandas as pd
df = pd.read_csv('output/spam_email.csv')

In [143]:
# spliting test and train samples
from sklearn.model_selection import train_test_split

df_predictors = df.drop('spam', axis = 1)
df_predicted = df['spam']

X_train, X_test, y_train, y_test = train_test_split(df_predictors, df_predicted)

In [153]:
# training the model 
from sklearn.linear_model import LogisticRegression

LRmodel = LogisticRegression(solver='lbfgs', max_iter=1600)
LRmodel.fit(X_train, y_train)

LogisticRegression(max_iter=1600)

In [155]:
# prediction score
LRmodel.score(X_test, y_test)

0.9235447437011295

The model is capable to detect in average 92.34% of spam emails;